In [4]:
import pickle
import random
import spacy
from spacy.util import minibatch, compounding

In [ ]:
train_data = None
test_data = None

In [ ]:
nlp = spacy.load("en_core_web_lg")

entity_linker = nlp.create_pipe("entity_linker", config={"incl_prior":False})
entity_linker.set_kb(kb)
nlp.add_pipe(entity_linker, last=True)

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "entity_linker"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for i in range(500):
        random.shuffle(train_data)#DOSTOSOWAĆ DANE
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,
                annotations
                drop=0.2,
                losses=losses,
                sgd=optimizer
            )
        if  i % 50 == 0:
            print(i, "Losses", losses)

nlp.to_disk(output_dir / "my_nlp_el")

with open(output_dir / "test_set.pkl", "wb") as f:
    pickle.dump(f)

In [ ]:
nlp = spacy.load(output_dir / "my_nlp_el")
text = ""
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_, ent.kb_id_)